# Reconocedor de billetes

In [32]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

In [33]:
denominaciones = ["Cincuenta", "Doscientos"]

masks = []
masks.append([[20, 0, 55], [100, 38, 136]]) #200
masks.append([[0, 0, 69], [106, 14, 158]]) #50

In [34]:
def funcion(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    ret, imgt = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    cv2.imshow("Image threshold", imgt)
    countours, hierarchy = cv2.findContours(imgt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rectangles = [cv2.boundingRect(countour) for countour in countours]
    sizeofImage = (400, 300)
    for rect in rectangles:
        if rect[2] > 100 and rect[3] > 50 and rect[2] < 620:
            imgn = img[rect[1]:rect[1] + rect[3], rect[0]:rect[0] + rect[2]]
            imgn = cv2.resize(imgn, (sizeofImage[0], sizeofImage[1]))
            
            imgn = cv2.cvtColor(imgn, cv2.COLOR_BGR2HSV)
            iWinner, p = 0, 0
            for i, mask in enumerate(masks):
                msk = cv2.inRange(imgn, np.array(mask[0]), np.array(mask[1]))
                iMsk = cv2.bitwise_and(imgn, imgn, mask=msk)
                iMsk = cv2.cvtColor(iMsk, cv2.COLOR_HSV2BGR)
                pOn = np.count_nonzero(iMsk != 0)
                if pOn > p:
                    p = pOn
                    iWinner = i
                    print("New Winner:", i, pOn)
            imgn = cv2.cvtColor(imgn, cv2.COLOR_HSV2BGR)            
            cv2.imshow("Image rect", imgn)

            cv2.rectangle(img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (255, 0, 0), 2)
            cv2.putText(img, denominaciones[iWinner], (rect[0], rect[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 0, 0), 3, cv2.LINE_AA)
    return img

In [30]:
masks = []

# A required callback method that goes into the trackbar function.
def nothing(x):
    pass

# Initializing the webcam feed.
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

# Create a window named trackbars.
cv2.namedWindow("Trackbars")

# Now create 6 trackbars that will control the lower and upper range of 
# H,S and V channels. The Arguments are like this: Name of trackbar, 
# window name, range,callback function. For Hue the range is 0-179 and
# for S,V its 0-255.

i, thearray = 0, []
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while True: 
    if len(masks) == len(denominaciones): break
    cv2.setTrackbarPos("L - H", "Trackbars", 0)
    cv2.setTrackbarPos("L - S", "Trackbars", 0)
    cv2.setTrackbarPos("L - V", "Trackbars", 0)
    cv2.setTrackbarPos("U - H", "Trackbars", 179)
    cv2.setTrackbarPos("U - S", "Trackbars", 255)
    cv2.setTrackbarPos("U - V", "Trackbars", 255)

    while True:

        # Start reading the webcam feed frame by frame.
        ret, frame = cap.read()
        if not ret: break
        # Flip the frame horizontally (Not required)
        frame = cv2.flip( frame, 1 ) 

        # Convert the BGR image to HSV image.
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Get the new values of the trackbar in real time as the user changes 
        # them
        l_h = cv2.getTrackbarPos("L - H", "Trackbars")
        l_s = cv2.getTrackbarPos("L - S", "Trackbars")
        l_v = cv2.getTrackbarPos("L - V", "Trackbars")
        u_h = cv2.getTrackbarPos("U - H", "Trackbars")
        u_s = cv2.getTrackbarPos("U - S", "Trackbars")
        u_v = cv2.getTrackbarPos("U - V", "Trackbars")

        # Set the lower and upper HSV range according to the value selected
        # by the trackbar
        lower_range = np.array([l_h, l_s, l_v])
        upper_range = np.array([u_h, u_s, u_v])

        # Filter the image and get the binary mask, where white represents 
        # your target color
        mask = cv2.inRange(hsv, lower_range, upper_range)

        # You can also visualize the real part of the target color (Optional)
        res = cv2.bitwise_and(frame, frame, mask=mask)

        # Converting the binary mask to 3 channel image, this is just so 
        # we can stack it with the others
        mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # stack the mask, orginal frame and the filtered result
        stacked = np.hstack((mask_3,frame,res))

        # Show this stacked frame at 40% of the size.
        cv2.imshow('Trackbars',cv2.resize(stacked,None,fx=0.4,fy=0.4))

        # If the user presses ESC then exit the program
        key = cv2.waitKey(1)
        if key == 27:
            break

        # If the user presses `s` then print this array.
        if key == ord('s'):

            thearray = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
            print(thearray)

            # Also save this array as penval.npy
            #np.save('hsv_value',thearray)
            
            break
        
    masks.append(thearray)

    
# Release the camera & destroy the windows.    
cap.release()
cv2.destroyAllWindows()

[[0, 0, 75], [108, 12, 158]]
[[0, 0, 53], [90, 30, 151]]


In [35]:
cam = cv2.VideoCapture(0)
while True:
    val, img = cam.read()
    img = funcion(img)
    cv2.imshow("Image",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

New Winner: 0 3576
New Winner: 1 8988
New Winner: 0 8514
New Winner: 1 22560
New Winner: 0 125484
New Winner: 0 7176
New Winner: 1 56421
New Winner: 0 7536
New Winner: 1 10122
New Winner: 0 179634
New Winner: 0 18933
New Winner: 1 81981
New Winner: 0 7863
New Winner: 1 12726
New Winner: 0 160068
New Winner: 0 194418
New Winner: 0 23862
New Winner: 1 123300
New Winner: 0 8580
New Winner: 1 14229
New Winner: 0 14025
New Winner: 1 30090
New Winner: 0 20133
New Winner: 1 41466
New Winner: 0 210534
New Winner: 0 268068
New Winner: 0 231456
New Winner: 0 232314
New Winner: 0 240438
New Winner: 0 36486
New Winner: 1 110319
New Winner: 0 233178
New Winner: 0 25479
New Winner: 1 131355
New Winner: 0 191211
New Winner: 0 225876
New Winner: 0 25479
New Winner: 1 131355
New Winner: 0 191211
New Winner: 0 225876
New Winner: 0 24084
New Winner: 1 122235
New Winner: 0 228360
New Winner: 0 20673
New Winner: 1 106692
New Winner: 0 215208
New Winner: 0 223911
New Winner: 0 235995
New Winner: 0 228849
Ne

New Winner: 1 9849
New Winner: 0 146610
New Winner: 1 177750
New Winner: 0 137775
New Winner: 1 214437
New Winner: 0 789
New Winner: 1 4437
New Winner: 0 151392
New Winner: 1 190203
New Winner: 0 134646
New Winner: 1 220278
New Winner: 0 708
New Winner: 1 5325
New Winner: 0 145020
New Winner: 1 183219
New Winner: 0 128004
New Winner: 1 218745
New Winner: 0 771
New Winner: 1 6243
New Winner: 0 140898
New Winner: 1 186861
New Winner: 0 129834
New Winner: 1 217791
New Winner: 0 1038
New Winner: 1 6300
New Winner: 0 143166
New Winner: 1 187878
New Winner: 0 128730
New Winner: 1 215946
New Winner: 0 1266
New Winner: 1 6813
New Winner: 0 146823
New Winner: 1 187503
New Winner: 0 128520
New Winner: 1 216411
New Winner: 0 975
New Winner: 1 5715
New Winner: 0 151554
New Winner: 1 186576
New Winner: 0 127614
New Winner: 1 216555
New Winner: 0 1290
New Winner: 1 6360
New Winner: 0 162720
New Winner: 1 189588
New Winner: 0 128217
New Winner: 1 217590
New Winner: 0 882
New Winner: 1 5976
New Winner

New Winner: 0 216300
New Winner: 0 47430
New Winner: 1 150438
New Winner: 0 212409
New Winner: 0 35082
New Winner: 1 127362
New Winner: 0 144129
New Winner: 1 147726
New Winner: 0 226278
New Winner: 0 37329
New Winner: 1 134982
New Winner: 0 124875
New Winner: 1 144549
New Winner: 0 222711
New Winner: 0 39990
New Winner: 1 131526
New Winner: 0 102174
New Winner: 1 129249
New Winner: 0 216096
New Winner: 0 55128
New Winner: 1 143511
New Winner: 0 101358
New Winner: 0 233898
New Winner: 0 56997
New Winner: 1 144078
New Winner: 0 98817
New Winner: 0 232317
New Winner: 0 28335
New Winner: 1 59868
New Winner: 0 100029
New Winner: 1 102642
New Winner: 0 228993
New Winner: 0 41178
New Winner: 1 61668
New Winner: 0 113514
New Winner: 1 116097
New Winner: 0 231198
New Winner: 0 133002
New Winner: 1 139458
New Winner: 0 242025
New Winner: 0 229065
New Winner: 0 227928
New Winner: 0 229875
New Winner: 0 226935
New Winner: 0 106653
New Winner: 0 214314
New Winner: 0 103572
New Winner: 0 208371
New

New Winner: 0 10848
New Winner: 1 50481
New Winner: 0 214017
New Winner: 0 12522
New Winner: 1 52569
New Winner: 0 212499
New Winner: 0 12285
New Winner: 1 55365
New Winner: 0 210213
New Winner: 0 13137
New Winner: 1 49512
New Winner: 0 207675
New Winner: 0 13137
New Winner: 1 49512
New Winner: 0 207675
New Winner: 0 9693
New Winner: 1 46986
New Winner: 0 206568
New Winner: 0 11862
New Winner: 1 49569
New Winner: 0 203340
New Winner: 0 9918
New Winner: 1 49344
New Winner: 0 216564
New Winner: 0 226656
New Winner: 0 10146
New Winner: 1 49716
New Winner: 0 204684
New Winner: 0 12006
New Winner: 1 51114
New Winner: 0 205308
New Winner: 0 13323
New Winner: 1 51006
New Winner: 0 205899
New Winner: 0 13344
New Winner: 1 52887
New Winner: 0 194349
New Winner: 0 9597
New Winner: 1 45141
New Winner: 0 206676
New Winner: 0 12834
New Winner: 1 56913
New Winner: 0 207879
New Winner: 0 10017
New Winner: 1 47898
New Winner: 0 216498
New Winner: 0 8538
New Winner: 1 39735
New Winner: 0 149496
New Win

New Winner: 0 138735
New Winner: 1 184719
New Winner: 0 163653
New Winner: 0 140379
New Winner: 1 183624
New Winner: 0 163653
New Winner: 0 140379
New Winner: 1 183624
New Winner: 0 163512
New Winner: 0 141633
New Winner: 1 185049
New Winner: 0 163515
New Winner: 0 135345
New Winner: 1 183516
New Winner: 0 159744
New Winner: 0 134061
New Winner: 1 181116
New Winner: 0 165138
New Winner: 0 134634
New Winner: 1 180015
New Winner: 0 160899
New Winner: 0 136188
New Winner: 1 179823
New Winner: 0 158673
New Winner: 0 138900
New Winner: 1 179217
New Winner: 0 164646
New Winner: 0 136380
New Winner: 1 178842
New Winner: 0 163323
New Winner: 0 133914
New Winner: 1 179052
New Winner: 0 155016
New Winner: 0 135216
New Winner: 1 175053
New Winner: 0 158097
New Winner: 0 134301
New Winner: 1 173976
New Winner: 0 158421
New Winner: 0 136281
New Winner: 1 168438
New Winner: 0 158421
New Winner: 0 136281
New Winner: 1 168438
New Winner: 0 160008
New Winner: 0 138348
New Winner: 1 162726
New Winner: 0

New Winner: 0 139761
New Winner: 1 167799
New Winner: 0 63456
New Winner: 1 182793
New Winner: 0 137787
New Winner: 1 165210
New Winner: 0 49827
New Winner: 1 161778
New Winner: 0 140694
New Winner: 1 205119
New Winner: 0 133290
New Winner: 1 150405
New Winner: 0 52182
New Winner: 1 160101
New Winner: 0 136179
New Winner: 1 205887
New Winner: 0 126576
New Winner: 1 142839
New Winner: 0 59052
New Winner: 1 173382
New Winner: 0 130530
New Winner: 1 201300
New Winner: 0 164679
New Winner: 1 198432
New Winner: 0 65295
New Winner: 1 182202
New Winner: 0 165861
New Winner: 1 201399
New Winner: 0 169827
New Winner: 1 209601
New Winner: 0 55101
New Winner: 1 164880
New Winner: 0 139833
New Winner: 1 204495
New Winner: 0 166782
New Winner: 1 206202
New Winner: 0 56637
New Winner: 1 166158
New Winner: 0 133518
New Winner: 1 199263
New Winner: 0 150264
New Winner: 1 198024
New Winner: 0 45609
New Winner: 1 139143
New Winner: 0 125094
New Winner: 1 196359
New Winner: 0 51273
New Winner: 1 150969
N

New Winner: 0 2748
New Winner: 1 43317
New Winner: 0 2322
New Winner: 1 43833
New Winner: 0 2259
New Winner: 1 44343
New Winner: 0 2220
New Winner: 1 44088
New Winner: 0 2511
New Winner: 1 44988
New Winner: 0 2508
New Winner: 1 44856
New Winner: 0 2613
New Winner: 1 45240
New Winner: 0 2613
New Winner: 1 45240
New Winner: 0 3117
New Winner: 1 45258
New Winner: 0 3162
New Winner: 1 45900
New Winner: 0 3192
New Winner: 1 45804
New Winner: 0 2517
New Winner: 1 14046
New Winner: 0 3120
New Winner: 1 46341
New Winner: 0 3117
New Winner: 1 46149
New Winner: 0 3204
New Winner: 1 47064
New Winner: 0 3492
New Winner: 1 47052
New Winner: 0 2361
New Winner: 1 13959
New Winner: 0 2394
New Winner: 1 13572
New Winner: 0 2427
New Winner: 1 13542
New Winner: 0 2427
New Winner: 1 13542
New Winner: 0 2256
New Winner: 1 13494
New Winner: 0 2421
New Winner: 1 13788
New Winner: 0 3126
New Winner: 1 45339
New Winner: 0 2811
New Winner: 1 44730
New Winner: 0 2682
New Winner: 1 43476
New Winner: 0 2757
New Wi